# Aphasia BCI Analysis

In [1]:
# imports
from pathlib import Path
import mne
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from copy import deepcopy

In [2]:
# load the data
def load_session(data_root: Path, prefix: str = 'AuditoryAphasia') -> mne.io.BaseRaw:
    """ Load BrainVision EEG data to mne.io.BaseRaw object """
    
    # Get all header files
    hdr_files = list(data_root.rglob(f'{prefix}*.vhdr'))
    hdr_files.sort()

    # read into single raw
    raws = [mne.io.read_raw_brainvision(hdrf) for hdrf in hdr_files]
    
    return mne.concatenate_raws(raws)

In [3]:
data_root = Path('./sub-VPpdfc/ses-230221/')
raws = load_session(data_root)
raws.info

Extracting parameters from sub-VPpdfc/ses-230221/AuditoryAphasia_pre_6d_250_0001.vhdr...
Setting channel info structure...
Extracting parameters from sub-VPpdfc/ses-230221/AuditoryAphasia_pre_6d_250_0002.vhdr...
Setting channel info structure...


<Info | 7 non-empty values
 bads: []
 ch_names: Fp2, F4, F8, FC6, C4, T8, CP2, CP6, TP10, P8, O2, PO10, P4, Cz, ...
 chs: 32 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-02-23 11:07:12 UTC
 nchan: 32
 projs: []
 sfreq: 500.0 Hz
>

In [4]:
# raws to epochs
# --------------------------------------------------------------------------
# from the aphasia repo, we know
# markers['target'] = [111,112,113,114,115,116]
# markers['nontarget'] = [101,102,103,104,105,106]
# markers['new-trial'] = [200,201,202,203,204,205]
markers = {'target': [111,112,113,114,115,116], 'nontarget': [101,102,103,104,105,106]}

def raws_to_epochs(raws: mne.io.BaseRaw, markers: dict = markers) -> mne.Epochs:
    ev, evid = mne.events_from_annotations(raws, verbose=False)

    # select the start events only
    sev = ev[np.isin(ev[:, 2], markers['target'] + markers['nontarget'])]

    # use mne's naming convention to be able to select targets and nontargets more easily
    names = {f'{k}/{e}': e for k, v in markers.items() for e in v}

    epo = mne.Epochs(raws, sev, event_id=names, tmin=-.2, tmax=1.2, verbose=False)
    
    return epo

In [5]:
# Preprocessing
epo = raws_to_epochs(raws)

# load data
epo.drop_bad()
epo.load_data()

# set montage
epo.set_montage("standard_1020")

# frequency filtering
epo.filter(0.5, 8)

Loading data for 1080 events and 701 original time points ...
0 bad epochs dropped
Loading data for 1080 events and 701 original time points ...
Setting up band-pass filter from 0.5 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 3301 samples (6.602 sec)



/var/folders/lq/l9pj97fx0xvc336klq1wkth40000gp/T/ipykernel_17989/3611833739.py:12: RuntimeWarning: filter_length (3301) is longer than the signal (701), distortion is likely. Reduce filter length or filter a longer signal.
  epo.filter(0.5, 8)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 34560 out of 34560 | elapsed:    4.1s finished


Number of events,1080
Events,nontarget/101: 150nontarget/102: 150nontarget/103: 150nontarget/104: 150nontarget/105: 150nontarget/106: 150target/111: 30target/112: 30target/113: 30target/114: 30target/115: 30target/116: 30
Time range,-0.200 – 1.200 sec
Baseline,-0.200 – 0.000 sec


In [6]:
# Create spatio temporal features from the epochs

# Note these are the default time windows we use in the Aphasia protocol, feel free to experiment
# with a different window set and see if you find a more optimal choice
twin = [[0.08, 0.15],
        [0.151, 0.21],
        [0.211, 0.28],
        [0.271, 0.35],
        [0.351, 0.44],
        [0.45, 0.56],
        [0.561, 0.7],
        [0.701, 0.85],
        [0.851, 1],
        [1.001, 1.2]]

# default channels set - also feel free to experiment with different channel sets
channels = ['Fp2', 'F4', 'F8', 'FC6', 'C4', 'T8', 'CP2', 'CP6', 'TP10', 'P8', 'O2', 'PO10',
            'P4', 'Cz', 'Pz', 'Oz', 'PO9', 'O1', 'P7', 'P3', 'CP5', 'CP1', 'T7', 'C3', 'FC1',
            'FCz', 'FC2', 'Fz', 'FC5', 'F7', 'F3', 'Fp1']

def epochs_to_features(epo: mne.BaseEpochs,
                       twin: list[list[float]] = twin,
                       channels: list[str] = channels,
                      ) -> np.ndarray:
    """
    Take the epochs object with data in shape (n_epochs, n_channels, n_times) and calculate 
    spatio temporal features for each epoch into a data matrix (n_epochs, n_features)
    """

    X = []
    n_epochs = len(epo)
    for tw in twin:
        X.append(
            epo.copy()
            .crop(*tw)
            .pick(channels)
            .get_data()
            .mean(axis=-1)   # mean accross time window
            .reshape(n_epochs, -1))
    X = np.hstack(X)
    return X

In [7]:
# Generate features and extract labels
X = epochs_to_features(epo)
y = np.asarray(
    [0 if e[-1] in markers['nontarget'] else 1 for e in epo.events]  # 0 for nontarget, 1 for target
)

In [116]:
# Build a classifier
clf = make_pipeline(
        LDA(solver='eigen', shrinkage='auto'),
    )

In [117]:
# test on a simple 80 / 20 split
ix = np.arange(len(y))
cutoff = int(.8 * len(ix))
ixtrain, ixtest = ix[:cutoff], ix[cutoff:]

clf.fit(X[ixtrain], y[ixtrain])
ypred = clf.predict(X[ixtest])

acc = accuracy_score(y[ixtest], ypred)
print(f"Accuracy in 80/20 split: {acc=}")

Accuracy in 80/20 split: acc=0.8425925925925926


In [119]:
# do a more proper cross validation
# from the way the paradigm was created, we know that we have 6 words which include one target word
# so a grouping in blocks of 6 words seems a natural way

assert len(epo) % 6 == 0, "The number of epochs is not a multiple of 6."\
" The data seems to include incomplete repetitions. Check which epochs belong to an incomplete block."

groups = np.arange(len(epo) // 6).repeat(6)

# check that there is exactly one target in each group
assert all([y[groups == g].sum() == 1 for g in np.unique(groups)]), "There are groups which do not have exactly"\
" one target epoch in them. Please investigate."
cv = GroupKFold(10)
print(f"Processing n={len(list(cv.split(X, y, groups=groups)))} splits during cross validation.")

scores = cross_val_score(clf, X, y, groups=groups, cv=cv, verbose=True)

print(f"Mean cross validation score: {np.mean(scores)}")

Processing n=10 splits during cross validation.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Mean cross validation score: 0.837037037037037


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.5s finished


In [128]:
type(cv)

sklearn.model_selection._split.GroupKFold

In [127]:
# Check the confusion matrix to get a better understanding of how the classifier works
y_pred = cross_val_predict(clf, X, y, groups=groups, cv=cv, verbose=False)
classes = [0, 1]
conf_mat = pd.DataFrame(
    confusion_matrix(y, y_pred, labels=classes), 
    columns=pd.Index(classes, name='PREDICTED'), 
    index=pd.Index(classes, name='TRUE'),
)
conf_mat

PREDICTED,0,1
TRUE,,
0,863,37
1,139,41


In [141]:
splits = list(cv.split(X, y, groups=groups))
ixtrain, ixtest = splits[0]

clf.fit(X[ixtrain], y[ixtrain])
ydesc = clf.decision_function(X[ixtest])

In [174]:
# As we see above, the confusion matrix reveals a problem of the classifier: We have quite a lot
# of false negatives. Lets see how our performance would be, if we where to classify at least one 
# tagert per group (whe have this information in the from the aphasia paper)

def aphasia_cross_val(clf: Pipeline, X: np.ndarray, y: np.ndarray, groups: np.ndarray,
                      cv: GroupKFold) -> list[float]:
    """ Perform a cross validation in which we enforce to decode on target per group """
    
    scores = []
    preds = np.zeros(len(y))  # store the prediction values from each fold to compute a confusion
                              # matrix later
    
    for ixtrain, ixtest in cv.split(X, y, groups):
        fold_clf = deepcopy(clf)
        fold_clf.fit(X[ixtrain], y[ixtrain])
        
        ypred = np.zeros(len(ixtest))
        # loop over the group indeces in the test epochs (each group is a tuple of 6 epochs)
        for g in np.unique(groups[ixtest]):
            gmask = groups[ixtest] == g
            decf = fold_clf.decision_function(X[ixtest][gmask])
            
            # choose only the position where the decision function is max. In case of a tie choose al
            ix = np.arange(len(ypred))[gmask][decf == decf.max()]
            ypred[ix] = 1
        
        # Calculate the accuracy of the predictions
        scores.append(accuracy_score(y[ixtest], ypred))
        preds[ixtest] = ypred

    return scores, preds

In [175]:
scores, preds = aphasia_cross_val(clf, X, y, groups=groups, cv=cv)

print(f"Mean cross validation score: {np.mean(scores)}")

classes = [0, 1]
conf_mat = pd.DataFrame(
    confusion_matrix(y, preds, labels=classes), 
    columns=pd.Index(classes, name='PREDICTED'), 
    index=pd.Index(classes, name='TRUE'),
)
conf_mat

Mean cross validation score: 0.8148148148148149


PREDICTED,0,1
TRUE,,
0,800,100
1,100,80
